In [4]:
import pandas as pd
import csv
import pyltr
import numpy as np
import datetime
import re
import time
suffixes = pd.read_csv("../data/Freq_background.csv", index_col='Unnamed: 0')
sortedpopular = pd.read_csv("../data/sorted_popular_queries.csv", index_col='Unnamed: 0')
with open('../data/ngram_dict.csv', mode='r') as infile:
    reader = csv.reader(infile)
    ngram_dict = {rows[0]:rows[1] for rows in reader}


/usr/lib/python3/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
prefix_suffix_pairs_background = pd.read_csv("../data/prefix_suffix_pairs.txt")

In [20]:
def get_single_ngram_features(query_input, ngram_dict):
    ngram_scores = []

    for n in range(1,7):
        sixgrams = ngrams(query_input.split(), n)
        ngram_score = 0
        for grams in sixgrams:
            try:
                ngram_score = ngram_score + float(ngram_dict[' '.join(grams)])
            except:
                pass
        ngram_scores.append(ngram_score)
    return ngram_scores

def get_ngram_features(inputs, ngram_dict):
    results = [get_single_ngram_features(x, ngram_dict) for x in inputs];
    return results


In [22]:
input_queries = ['hello my name is jeff', 'hello my name is juan']

resultvalue = get_ngram_features(input_queries, ngram_dict)
resultvalue

NameError: name 'ngrams' is not defined